# StateLog × TradeLog 解析テンプレート
- `analysis/env_data` と `analysis/bt_results` に配置した CSV を読み込み、環境指標とBT結果を突き合わせるためのノートです。
- `merged_trades` DataFrame を作成しておけば、帯域別・時間別の集計を容易に反復できます。

In [1]:
import pandas as pd
from pathlib import Path
from functools import lru_cache


def find_workspace_root(marker='analysis'):
    current = Path.cwd().resolve()
    for candidate in [current, *current.parents]:
        if (candidate / marker).is_dir():
            return candidate
    raise FileNotFoundError("'{}' directory not found from {}".format(marker, current))


WORKSPACE_ROOT = find_workspace_root()
DATA_ROOT = WORKSPACE_ROOT / 'analysis'
ENV_DIR = DATA_ROOT / 'env_data'
BT_DIR = DATA_ROOT / 'bt_results'


def list_csv(directory, pattern='*.csv'):
    return sorted(directory.glob(pattern))


print('Workspace root:', WORKSPACE_ROOT)
print('Env dir:', ENV_DIR)
print('BT dir:', BT_DIR)
print('Detected env csv:', len(list_csv(ENV_DIR)))
print('Detected bt csv:', len(list_csv(BT_DIR, 'TradeLog_*.csv')))


def resolve_bt_path(bt_name=None, pattern='TradeLog_*.csv'):
    if bt_name:
        candidate = BT_DIR / bt_name
        if candidate.exists():
            return candidate
        raise FileNotFoundError(f"{candidate} が存在しません")
    files = list_csv(BT_DIR, pattern)
    if not files:
        raise FileNotFoundError(f"BT_DIR に {pattern} が見つかりません")
    return files[-1]


def load_bt_dataframe(bt_name=None):
    bt_path = resolve_bt_path(bt_name)
    df = pd.read_csv(bt_path)
    df['timestamp'] = pd.to_datetime(df['timestamp'].str.replace('.', '-', regex=False))
    return df, bt_path


@lru_cache(maxsize=1)
def build_env_index():
    env_map = {}
    for csv_path in list_csv(ENV_DIR, '*.csv'):
        digits = ''.join(filter(str.isdigit, csv_path.stem))
        if len(digits) < 8:
            continue
        key = digits[-8:]
        env_map.setdefault(key, []).append(csv_path)
    return {k: tuple(v) for k, v in env_map.items()}


def load_env_frames(date_keys, sep=';'):
    env_map = build_env_index()
    frames = []
    missing = []
    for key in date_keys:
        paths = env_map.get(key)
        if not paths:
            missing.append(key)
            continue
        day_frames = [pd.read_csv(path, sep=sep) for path in paths]
        env_part = pd.concat(day_frames, ignore_index=True)
        env_part['bar_time'] = pd.to_datetime(env_part['bar_time'].str.replace('.', '-', regex=False))
        frames.append(env_part)
    return frames, missing


def concat_env_frames(frames):
    if not frames:
        raise ValueError('frames is empty')
    return pd.concat(frames, ignore_index=True).sort_values('bar_time')


def merge_entries_with_env(bt_df, env_df, tolerance='5min'):
    entries = bt_df[bt_df['event'] == 'ENTRY'].copy().sort_values('timestamp')
    env_sorted = env_df.sort_values('bar_time')
    merged = pd.merge_asof(entries, env_sorted, left_on='timestamp', right_on='bar_time',
                           direction='nearest', tolerance=pd.Timedelta(tolerance))
    return merged


def build_entry_exit(bt_df, merged):
    exits = bt_df[bt_df['event'] == 'EXIT'][['ticket','net','pips','timestamp']].copy()
    exits = exits.rename(columns={'timestamp':'exit_timestamp','net':'exit_net','pips':'exit_pips'})
    entry_exit = merged.merge(exits, on='ticket', how='left')
    return entry_exit


def ensure_entry_exit():
    notebook_globals = globals()
    entry_exit = notebook_globals.get('entry_exit')
    if entry_exit is not None:
        return entry_exit
    bt_df = notebook_globals.get('bt_df')
    merged = notebook_globals.get('merged')
    if bt_df is None or merged is None:
        raise NameError('entry_exit を生成するには bt_df と merged が必要です。')
    entry_exit = build_entry_exit(bt_df, merged)
    notebook_globals['entry_exit'] = entry_exit
    return entry_exit


Workspace root: /home/anyo_/workspace/YoYoEA_Multi_Entry
Env files: [PosixPath('/home/anyo_/workspace/YoYoEA_Multi_Entry/analysis/env_data/StateLog_USDJPY_M5_20250509.csv'), PosixPath('/home/anyo_/workspace/YoYoEA_Multi_Entry/analysis/env_data/StateLog_USDJPY_M5_20250929.csv'), PosixPath('/home/anyo_/workspace/YoYoEA_Multi_Entry/analysis/env_data/StateLog_USDJPY_M5_20251028.csv'), PosixPath('/home/anyo_/workspace/YoYoEA_Multi_Entry/analysis/env_data/StateLog_USDJPY_M5_20250125.csv'), PosixPath('/home/anyo_/workspace/YoYoEA_Multi_Entry/analysis/env_data/StateLog_USDJPY_M5_20250505.csv'), PosixPath('/home/anyo_/workspace/YoYoEA_Multi_Entry/analysis/env_data/StateLog_USDJPY_M5_20250916.csv'), PosixPath('/home/anyo_/workspace/YoYoEA_Multi_Entry/analysis/env_data/StateLog_USDJPY_M5_20250513.csv'), PosixPath('/home/anyo_/workspace/YoYoEA_Multi_Entry/analysis/env_data/StateLog_USDJPY_M5_20251111.csv'), PosixPath('/home/anyo_/workspace/YoYoEA_Multi_Entry/analysis/env_data/StateLog_USDJPY_M5_20

In [ ]:
# ==== 自動で環境データを取得して突き合わせる ====
BT_FILE_NAME = None  # 解析対象のBTファイルを指定する場合はファイル名を入力

bt_df, bt_path = load_bt_dataframe(BT_FILE_NAME)
print(f'Using BT file: {bt_path.name} ({len(bt_df)} rows)')

date_keys = sorted(bt_df['timestamp'].dt.strftime('%Y%m%d').unique())
env_frames, missing_dates = load_env_frames(tuple(date_keys))
if missing_dates:
    print('[WARN] 環境ファイル未検出日:', ', '.join(missing_dates))

if not env_frames:
    raise FileNotFoundError('BT対象日に対応する環境ファイルが見つかりません')

env_df = concat_env_frames(env_frames)
print('Loaded env rows:', len(env_df), 'from', len(env_frames), '日分')

merged = merge_entries_with_env(bt_df, env_df)
print('merged rows', len(merged))
merged.head()


Loaded env rows: 15626 from 56 filesets
merged rows 4250


,timestamp_x,run_id_x,event,symbol_x,profile_x,strategy,direction,ticket,volume,price,...,ma_long,ma_slope,bb_width,donchian_width,fibo_ratio,spread,session,weekday,volatility,notes
0,2025-09-01 01:05:00,20250901_010100,ENTRY,USDJPY,AtrBandConfig_YoYoEA_Multi_Entry_Test_ADXAdapt...,MACD,SELL,1,0.1,146.946,...,147.031650,-0.102,0.0,0.199,0.497487,0.5,ASIA,1,LOW,NaN
1,2025-09-01 02:00:00,20250901_010100,ENTRY,USDJPY,AtrBandConfig_YoYoEA_Multi_Entry_Test_ADXAdapt...,MACD,BUY,2,0.1,147.020,...,147.035225,0.187,0.0,0.177,0.661017,0.5,ASIA,1,LOW,NaN
2,2025-09-01 02:05:00,20250901_010100,ENTRY,USDJPY,AtrBandConfig_YoYoEA_Multi_Entry_Test_ADXAdapt...,CCI,BUY,3,0.1,147.058,...,147.035770,0.252,0.0,0.178,0.764045,0.5,ASIA,1,LOW,NaN
3,2025-09-01 02:15:00,20250901_010100,ENTRY,USDJPY,AtrBandConfig_YoYoEA_Multi_Entry_Test_ADXAdapt...,CCI,BUY,4,0.1,147.061,...,147.036615,0.295,0.0,0.178,0.752809,0.5,ASIA,1,LOW,NaN
4,2025-09-01 02:40:00,20250901_010100,ENTRY,USDJPY,AtrBandConfig_YoYoEA_Multi_Entry_Test_ADXAdapt...,MA_CROSS,BUY,5,0.1,147.089,...,147.038675,0.365,0.0,0.178,0.837079,0.5,ASIA,1,LOW,NaN


In [3]:
# ==== ENTRY環境とEXIT結果を結合 ====
entry_exit = build_entry_exit(bt_df, merged)
print('combined rows', len(entry_exit))
entry_exit.head()


combined rows 4250


,timestamp_x,run_id_x,event,symbol_x,profile_x,strategy,direction,ticket,volume,price,...,donchian_width,fibo_ratio,spread,session,weekday,volatility,notes,exit_net,exit_pips,exit_timestamp
0,2025-09-01 01:05:00,20250901_010100,ENTRY,USDJPY,AtrBandConfig_YoYoEA_Multi_Entry_Test_ADXAdapt...,MACD,SELL,1,0.1,146.946,...,0.199,0.497487,0.5,ASIA,1,LOW,NaN,-5.71,-8.4,2025-09-01 01:06:28
1,2025-09-01 02:00:00,20250901_010100,ENTRY,USDJPY,AtrBandConfig_YoYoEA_Multi_Entry_Test_ADXAdapt...,MACD,BUY,2,0.1,147.020,...,0.177,0.661017,0.5,ASIA,1,LOW,NaN,1.36,2.0,2025-09-01 02:02:40
2,2025-09-01 02:05:00,20250901_010100,ENTRY,USDJPY,AtrBandConfig_YoYoEA_Multi_Entry_Test_ADXAdapt...,CCI,BUY,3,0.1,147.058,...,0.178,0.764045,0.5,ASIA,1,LOW,NaN,1.36,2.0,2025-09-01 02:08:16
3,2025-09-01 02:15:00,20250901_010100,ENTRY,USDJPY,AtrBandConfig_YoYoEA_Multi_Entry_Test_ADXAdapt...,CCI,BUY,4,0.1,147.061,...,0.178,0.752809,0.5,ASIA,1,LOW,NaN,1.36,2.0,2025-09-01 02:41:46
4,2025-09-01 02:40:00,20250901_010100,ENTRY,USDJPY,AtrBandConfig_YoYoEA_Multi_Entry_Test_ADXAdapt...,MA_CROSS,BUY,5,0.1,147.089,...,0.178,0.837079,0.5,ASIA,1,LOW,NaN,1.36,2.0,2025-09-01 02:49:47


In [4]:
# ==== 低ATR帯の条件別集計 ====
entry_exit = ensure_entry_exit()
entry_exit['atr_entry'] = entry_exit['atr_entry'].astype(float)
low = entry_exit[entry_exit['atr_entry'] <= 0.10].copy()
if low.empty:
    raise ValueError('低ATR帯のデータが存在しません')
if 'session' not in low.columns:
    low['session'] = 'UNKNOWN'
else:
    low['session'] = low['session'].fillna('UNKNOWN')
low['adx_bucket'] = pd.cut(low['adx14'], bins=[0,15,20,25,100], labels=['<=15','15-20','20-25','25+'], right=False)
low['donchian_bucket'] = pd.cut(low['donchian_width'], bins=[0,0.15,0.25,1], labels=['narrow','mid','wide'], right=False)
summary = low.pivot_table(index=['session','adx_bucket','donchian_bucket'],
                        values='exit_net', aggfunc=['count','sum','mean'], fill_value=0)
summary.sort_values(('sum','exit_net'), ascending=True).head(15)


/tmp/ipykernel_113746/1913019154.py:12: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  summary = low.pivot_table(index=['session','adx_bucket','donchian_bucket'],
/tmp/ipykernel_113746/1913019154.py:12: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  summary = low.pivot_table(index=['session','adx_bucket','donchian_bucket'],
/tmp/ipykernel_113746/1913019154.py:12: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  summary = low.pivot_table(index=['session','adx_bucket','donchian_bucket'],


count      sum      mean
                                   exit_net exit_net  exit_net
session adx_bucket donchian_bucket                            
US      25+        wide                 586  -347.30 -0.592662
EUROPE  25+        mid                  336  -125.96 -0.374881
US      25+        mid                  265  -105.82 -0.399321
        15-20      mid                   48  -105.17 -2.191042
                   wide                  49  -101.89 -2.079388
EUROPE  25+        wide                 722   -69.80 -0.096676
OTHER   25+        narrow                55   -68.12 -1.238545
EUROPE  20-25      mid                   64   -64.32 -1.005000
ASIA    15-20      mid                   34   -42.57 -1.252059
EUROPE  20-25      wide                  60   -41.55 -0.692500
        25+        narrow                42   -38.51 -0.916905
        15-20      mid                   38   -30.79 -0.810263
OTHER   25+        mid                  110   -29.35 -0.266818
ASIA    20-25      narrow                63   -28.24 -0.448254
                   wide                  56   -22.96 -0.410000

In [5]:
# ==== ATR×ADX帯域の損益集計 ====
entry_exit = ensure_entry_exit()
attr = entry_exit.dropna(subset=['atr_entry','adx14']).copy()
if attr.empty:
    raise ValueError('ATRもしくはADXが不足しています')
attr['atr_bucket'] = pd.cut(attr['atr_entry'].astype(float), bins=[0,0.05,0.08,0.10,0.12,0.20,1], labels=['0.00-0.05','0.05-0.08','0.08-0.10','0.10-0.12','0.12-0.20','0.20+'], right=False)
attr['adx_bucket'] = pd.cut(attr['adx14'], bins=[0,15,20,25,30,100], labels=['<=15','15-20','20-25','25-30','30+'], right=False)
summary = attr.pivot_table(index=['atr_bucket','adx_bucket'], values='exit_net', aggfunc=['count','sum','mean'], fill_value=0)
summary


/tmp/ipykernel_113746/1365422209.py:9: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  summary = attr.pivot_table(index=['atr_bucket','adx_bucket'], values='exit_net', aggfunc=['count','sum','mean'], fill_value=0)
/tmp/ipykernel_113746/1365422209.py:9: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  summary = attr.pivot_table(index=['atr_bucket','adx_bucket'], values='exit_net', aggfunc=['count','sum','mean'], fill_value=0)
/tmp/ipykernel_113746/1365422209.py:9: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
 

count      sum       mean
                      exit_net exit_net   exit_net
atr_bucket adx_bucket                             
0.00-0.05  <=15             23   -26.03  -1.131739
           15-20           150     7.90   0.052667
           20-25           254  -111.79  -0.440118
           25-30           378   -14.52  -0.038413
           30+            1223  -268.78  -0.219771
0.05-0.08  <=15             15   -17.60  -1.173333
           15-20           128  -133.57  -1.043516
           20-25           211   -13.04  -0.061801
           25-30           257  -322.43  -1.254591
           30+            1041   -66.40  -0.063785
0.08-0.10  <=15              3    26.45   8.816667
           15-20            34    -6.73  -0.197941
           20-25            60    45.88   0.764667
           25-30            43   -50.09  -1.164884
           30+             230   -75.79  -0.329522
0.10-0.12  <=15              2     1.71   0.855000
           15-20             4   -15.60  -3.900000
           20-25            16   -19.09  -1.193125
           25-30            24   -38.49  -1.603750
           30+              65    57.42   0.883385
0.12-0.20  <=15              4  -124.59 -31.147500
           15-20             6   -24.34  -4.056667
           20-25            10     5.00   0.500000
           25-30             8    15.68   1.960000
           30+              53   233.24   4.400755
0.20+      <=15              0     0.00        NaN
           15-20             0     0.00        NaN
           20-25             2   -13.60  -6.800000
           25-30             3    40.27  13.423333
           30+               3    25.17   8.390000

In [6]:
# ==== ATR×ADX×ストラテジーの損益 ====
entry_exit = ensure_entry_exit()
attr = entry_exit.dropna(subset=['atr_entry','adx14']).copy()
attr['atr_bucket'] = pd.cut(attr['atr_entry'].astype(float), bins=[0,0.05,0.08,0.10,0.12,0.20,1], labels=['0.00-0.05','0.05-0.08','0.08-0.10','0.10-0.12','0.12-0.20','0.20+'], right=False)
attr['adx_bucket'] = pd.cut(attr['adx14'], bins=[0,15,20,25,30,100], labels=['<=15','15-20','20-25','25-30','30+'], right=False)
table = attr.pivot_table(index=['atr_bucket','adx_bucket','strategy'], values='exit_net', aggfunc=['count','sum','mean'], fill_value=0)
table = table.reset_index()
table[['count','sum','mean']] = table[['count','sum','mean']].round(2)
table


/tmp/ipykernel_113746/1420206658.py:7: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  table = attr.pivot_table(index=['atr_bucket','adx_bucket'], columns='strategy', values='exit_net', aggfunc=['count','sum','mean'], fill_value=0)
/tmp/ipykernel_113746/1420206658.py:7: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  table = attr.pivot_table(index=['atr_bucket','adx_bucket'], columns='strategy', values='exit_net', aggfunc=['count','sum','mean'], fill_value=0)
/tmp/ipykernel_113746/1420206658.py:7: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warni

count                              sum                   \
strategy                CCI MACD MA_CROSS  RSI STOCH     CCI    MACD MA_CROSS   
atr_bucket adx_bucket                                                           
0.00-0.05  <=15          13    9        1    0     0  -17.31   -3.69    -5.03   
           15-20         62   60        5    2    21  -13.76   19.70     1.39   
           20-25        123   78        9    9    35  -68.10  -39.75     1.07   
           25-30        180  103       12   31    52   -4.09   24.99    -9.73   
           30+          589  146       39  177   272 -133.11   35.48   -15.09   
0.05-0.08  <=15           6    6        0    0     3  -19.17  -11.98     0.00   
           15-20         45   52        6    3    22 -113.42  -19.12     3.00   
           20-25         94   71        7    5    34  -20.43  -53.47    -1.05   
           25-30        128   58        8   24    39 -247.14  -32.54   -41.95   
           30+          435  148       38  198   222  210.79  -24.80   -22.42   
0.08-0.10  <=15           0    3        0    0     0    0.00   26.45     0.00   
           15-20         17   10        2    2     3  -30.07   -5.73     3.99   
           20-25         23   23        2    5     7   24.24   35.07   -13.33   
           25-30         17   16        0    3     7  -30.71  -18.51     0.00   
           30+           94   35       10   41    50   37.21   -4.42     2.52   
0.10-0.12  <=15           0    0        0    1     1    0.00    0.00     0.00   
           15-20          1    1        0    0     2  -20.35   18.75     0.00   
           20-25          5    6        0    1     4   16.99  -21.03     0.00   
           25-30         10    3        3    2     6  -23.29   10.58    -4.19   
           30+           21   10        1   13    20   33.43  105.09    16.72   
0.12-0.20  <=15           1    1        1    0     1  -33.79  -31.72   -31.55   
           15-20          2    2        1    1     0  -23.79   28.61     3.40   
           20-25          4    1        1    2     2   73.92  -37.58   -32.34   
           25-30          2    2        0    1     3   -6.77  -23.95     0.00   
           30+           17   12        2   11    11 -128.79  -46.08    53.96   
0.20+      <=15           0    0        0    0     0    0.00    0.00     0.00   
           15-20          0    0        0    0     0    0.00    0.00     0.00   
           20-25          0    1        0    0     1    0.00  -47.59     0.00   
           25-30          0    1        1    0     1    0.00   36.56    44.70   
           30+            1    1        0    0     1   45.71   38.11     0.00   

                                            mean                        \
strategy                  RSI   STOCH        CCI       MACD   MA_CROSS   
atr_bucket adx_bucket                                                    
0.00-0.05  <=15          0.00    0.00  -1.331538  -0.410000  -5.030000   
           15-20         7.86   -7.29  -0.221935   0.328333   0.278000   
           20-25       -10.52    5.51  -0.553659  -0.509615   0.118889   
           25-30        -2.90  -22.79  -0.022722   0.242621  -0.810833   
           30+         -65.92  -90.14  -0.225993   0.243014  -0.386923   
0.05-0.08  <=15          0.00   13.55  -3.195000  -1.996667   0.000000   
           15-20        16.05  -20.08  -2.520444  -0.367692   0.500000   
           20-25         1.55   60.36  -0.217340  -0.753099  -0.150000   
           25-30        -6.73    5.93  -1.930781  -0.561034  -5.243750   
           30+         -97.35 -132.62   0.484575  -0.167568  -0.590000   
0.08-0.10  <=15          0.00    0.00   0.000000   8.816667   0.000000   
           15-20        -4.88   29.96  -1.768824  -0.573000   1.995000   
           20-25        -9.53    9.43   1.053913   1.524783  -6.665000   
           25-30       -11.42   10.55  -1.806471  -1.156875   0.000000   
           30+         -43.68  -67.42   0.395851  -0.126286   0.252000   
0.10-0.12  <=15 

In [ ]:
# ==== Donchian×ATR×ストラテジーの損益 ====
entry_exit = ensure_entry_exit()
don = entry_exit.dropna(subset=['atr_entry','donchian_width']).copy()
don['atr_bucket'] = pd.cut(don['atr_entry'].astype(float), bins=[0,0.05,0.08,0.10,0.12,0.20,1], labels=['0.00-0.05','0.05-0.08','0.08-0.10','0.10-0.12','0.12-0.20','0.20+'], right=False)
don['donchian_bucket'] = pd.cut(don['donchian_width'], bins=[0,0.15,0.25,0.4,1], labels=['narrow','mid','wide','ultra'], right=False)
table_d = don.pivot_table(index=['atr_bucket','donchian_bucket','strategy'], values='exit_net', aggfunc=['count','sum','mean'], fill_value=0)
table_d = table_d.reset_index()
table_d[['count','sum','mean']] = table_d[['count','sum','mean']].round(2)
table_d


In [7]:
# ==== 例: ATR帯域×戦略の損益集計 ====
exits = bt_df[bt_df['event'] == 'EXIT'].copy()
exits['atr_entry'] = exits['atr_entry'].astype(float)
bins = [0, 0.05, 0.08, 0.10, 0.12, 0.20, 1]
labels = ['0.00-0.05','0.05-0.08','0.08-0.10','0.10-0.12','0.12-0.20','0.20+']
exits['atr_band'] = pd.cut(exits['atr_entry'], bins=bins, labels=labels, right=False)
pivot = exits.pivot_table(index='atr_band', columns='strategy', values='net', aggfunc='sum', fill_value=0)
pivot

/tmp/ipykernel_113746/318773033.py:7: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot = exits.pivot_table(index='atr_band', columns='strategy', values='net', aggfunc='sum', fill_value=0)


strategy,CCI,MACD,MA_CROSS,RSI,STOCH
atr_band,,,,,
0.00-0.05,-236.37,36.73,-27.39,-71.48,-114.71
0.05-0.08,-189.37,-141.91,-62.42,-86.48,-72.86
0.08-0.10,0.67,32.86,-6.82,-69.51,-17.48
0.10-0.12,6.78,113.39,12.53,-53.96,-92.79
0.12-0.20,-119.22,-110.72,-6.53,119.15,222.31
0.20+,45.71,27.08,44.70,0.00,-65.65


## TODO
- 週次/日次での State×BT 結合関数を関数化する
- 追加の特徴量（ATR増減、セッションフラグなど）を派生列として定義する
- ML モデル用に `merged` から特徴量テーブルを作成する